In [ ]:
'''1.导入pandas库'''
import pandas as pd

'''2.导入文件'''
#设置文件名称
file="运费明细表.xlsx"
df = pd.read_excel(file,sheet_name='运费明细',skiprows=3,header=2,usecols='C:F',dtype={'ID':object,'账单日期':str,'出口日期':str},index_col='ID')
#skiprows 跳过前3行
#header 从第3行开始读
#usecols 列选择C到F列数据读取
#dtype 重点 pandas把NaN默认flode 如果想下面迭代表达先把空列设置成str类型、或者object
#index_col 把ID列作为DateFrame的index列
#sheet_name=‘运费明细’ 把Excel表中读取运费明细表 或者sheet2

'''3.导入CSV,TSV,TXT文件中的数据'''
##导入csv文件
df = pd.read_csv(file, index_col='ID')
##导入Tsv文件，文件中带有\t分隔符
df = pd.read_csv(file, sep='\t', index_col='ID')
##导入TXT文件，文件中带有'|'分隔符
df = pd.read_csv(file, sep='|', index_col='ID')
#读取文件endcoding为简体中文格式，从第二行开始读
df = pd.read_csv(file,encoding='gb18030',header=1) 


'''4.保存文件'''
##保存为EXE文件
df.to_excel(file)
##保存为CSV文件
df.to_csv(file,encoding='gb18030')
#如果未设置index列系统自动保存，excel打开后 多出一列index，解决方案指定index列
#方法：
df=df.set_index('ID')
df.set_index('ID'，inplace=True)


'''5.排序多重排序'''
#先以 账单日期按顺序排序，再以出口日期按倒序排序
#ascending True 从小到大
#inplace True 直接在df数据上保留修改
df.sort_values(by=['账单日期', '出口日期'], ascending=[True,False], inplace=True)
##ascending False：倒序,从大到小
df.sort_values(by='出口日期',inplace=True,ascending=False)


'''6.数据查看'''
#查看每列数据数据类型 str time object等等
df.info()
#查看头部前三行
df.head(3)
#查看尾部
df.tail()


'''7.多表合并'''
##df1为需合并表格，df2为被合并表格，将两表合并
table = df1.merge(df2,how='left',on='ID').fillna('没找到')
##或者
table=pd.merge(df1,df2,on='ID')
#how=’left‘ 表示依 df1 基础 保留所有df1列信息。默认inner参数
#on=’ID‘ df1与df2都有ID列 前提两张表都有ID列，没有用 left_on与right_on
#.fillna() 表示 在df1中df2没有的数据填下’没找到‘
#merge() 不能默认指定index列 必须 on指定
#how='outer'外连接：并集
#how='inner"内连接：交集
#how='left'左连接：左边对象全部保留
#how='right'右连接：右边对象全部保留

#用join 合并指定表格
table = df1.join(df2, how='left').fillna(0)


'''8.分类汇总'''
#分类方法：
groups = df.groupby(['账单日期', '出口日期'])
#根据列分组
s = groups['应收费用'].sum()
#ID列求计数
c = groups['ID'].count()
#两两合并
df2 = pd.DataFrame({'Sum': s, 'Count': c})
#s、c 两个DataFrame 按照列Sum 与 Count 再合并成一个新DateFrame


'''9.消除重复数据'''
##将账单号码这一列重复的消除，并保留第一个,其参数为keep='first'
##当keep='last'时就是保留最后一次出现的重复行。 
##inplace=True它将从原始DataFrame中删除所有重复。
df=df.drop_duplicates(subset='账单号码', inplace=True, keep='first')
print(df)


'''10.删除行列方法'''
#删除ID，账单日期两列
df.drop(["ID","账单日期"],axis =1)
##或
df.drop(columns = ["ID","账单日期"])
##删除行的方法
df.drop(["0a","1b"],axis = 0)
#或
df.drop(df.index[[0,1]])
##删除特定的行，将ID大于40的行列出来
df[df["ID"]<40]


'''11.旋转数据表'''
##行列转换
table = df.transpose()
##智能将行列转秩
df=df.T

'''12.列与列的运算'''
##算术相加
df=df["实际重量"]+df["计费重量"]
##相减
df=df["实际重量"]-df["计费重量"]
##相乘
df=df["实际重量"]*df["计费重量"]
##相除
df=df["实示重量"]/df["计费重量"]
##求平均
df=df["实示重量"].mean()
##求合计
df=df["实示重量"].sum()

'''13.单列折分'''
##split的意思是折分，‘：’是折份的标准，一列变成两列
df = df['运单号（长）'].str.split('：', expand=True)
##或者用自动义函数
def split_func(line):
    line["运单号"], line["长"] = line["运单号（长）"].split(":")
    return line
df = df.apply(split_func, axis=1)

